# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Ämter der Bischöfe einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: `bischof.tbl_bischoefe_gatz` und `bischof.tbl_bischoefe_gatzaemter`

[Amt](#Amt)  
[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt)





Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path = "../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

*2022-01-13* Für die offizielle Liste der Ämter gibt es inzwischen eine Liste: `domherr.tbl_aemterliste_normiert`. Die Ämter der Bischöfe sind neu zuzuordnen.

## Amtsperiode

In [10]:
sql = "SELECT * FROM bischof.tbl_bischoefe_gatzaemter"
df_pr = Wds.sql_df(sql);

In [11]:
size(df_pr)

(7329, 15)

In [12]:
names(df_pr)

15-element Vector{String}:
 "Bistum"
 "Amtsart"
 "Amtsbeginn"
 "Amtsende"
 "ID_Amt"
 "ID_Bischof"
 "Institution"
 "Amt_weltlich"
 "Herrschaftsgebiet"
 "Ort"
 "Amt_Kommentar"
 "Sort"
 "Titularbistum"
 "Profession"
 "id_gsn"

nicht befüllt:
- Herrschafsgebiet
- Institution
- Amt_weltlich
- Profession
- Ort

### Bischöfe zuordnen

In [13]:
transform!(df_pr, :ID_Bischof => ByRow(string) => :id_in_source);

In [14]:
bishop_item_type = 4;
sql = "SELECT id AS person_id, id_in_source FROM item WHERE item_type_id = $(bishop_item_type)";
df_idx = Wds.sql_df(sql);

In [15]:
df_prp = leftjoin(df_pr, df_idx, on = :id_in_source);

In [16]:
size(df_prp)

(7329, 17)

Amtsangaben ohne Bezug zu einem Bischof

In [17]:
sort_ID_Bischof(df) = sort(df, :ID_Bischof)
filter(:person_id => ismissing, df_prp) |> sort_ID_Bischof

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution
,String?,String?,String?,String?,Int32?,Int32?,String?
1,Brixen,Weihbischof,1379,1387,22075,21066,missing
2,Chur,Weihbischof,um 1289,1294,22087,21078,missing
3,Eichstätt,Weihbischof,1340,missing,22102,21093,missing
4,Halberstadt,Weihbischof,1420,1436,22144,21135,missing
5,Köln,Weihbischof,1228,missing,22173,21164,missing
6,Köln,Weihbischof,1352,1364,22184,21175,missing
7,Konstanz,Weihbischof,1289,1294 (1299 ?),22198,21189,missing
8,Leitomischl,Weihbischof,1361,missing,22238,21229,missing
9,Lüttich,Weihbischof,1233,missing,22243,21234,missing


19 Amtsperioden können keinem Bischof zugeordnet werden?!

In [17]:
df_prp = dropmissing(df_prp, :person_id);

In [18]:
size(df_prp)

(7310, 17)

Lösche Einträge ohne Amtsbezeichnung

In [19]:
df_prp = dropmissing(df_prp, :Amtsart);

In [20]:
size(df_prp)

(7308, 17)

### Rollen/Ämter zuordnen

In [21]:
sql = "SELECT name, id FROM role"
df_role = Wds.sql_df(sql);

In [22]:
lookup_role = Dict(df_role.name .=> df_role.id);

In [23]:
get_role_id(name) = get(lookup_role, name, missing)

get_role_id (generic function with 1 method)

In [24]:
get_role_id("Vikar")

74166

Die Einträge in Tabelle `role` beginnen alle mit einem Großbuchstaben

In [25]:
starts_lowercase(s) = islowercase(s[1]);
df_prp_sl = filter(:Amtsart => starts_lowercase, df_prp);

In [26]:
select(df_prp_sl, [:person_id, :id_in_source, :Amtsart, :Bistum])

,person_id,id_in_source,Amtsart,Bistum
,Int32,String,String,String?
1,2954,10429,nominierter Bischof,Lüttich
2,2693,4254,designierter Bischof,Konstanz
3,5247,21771,evangelischer Administrator,Bremen-Hamburg
4,5248,21772,evangelischer Administrator,Bremen-Hamburg
5,5249,21773,evangelischer Administrator,Bremen-Hamburg


In [27]:
transform!(df_prp, :Amtsart => ByRow(uppercasefirst) => :Amtsart);

SQL berücksichtigt keine Unterschiede in Groß- und Kleinschreibung

In [28]:
df_info = subset(df_prp, :ID_Bischof => ByRow(isequal(10127)))

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof
,String?,String,String?,String?,Int32?,Int32?
1,Münster,Fürstbischof,1719,1761,5609,10127
2,Regensburg,Fürstbischof,1717,1719,4222,10127
3,Paderborn,Fürstbischof,1719,1761,4224,10127
4,Köln,Koadjutor des Erzbischofs,1722,1723,4225,10127
5,Köln,Kurfürst-Erzbischof,1723,1761,4226,10127
6,Hildesheim,Fürstbischof,1724,1761,4227,10127
7,Osnabrück,Fürstbischof,1728,1761,4228,10127
8,missing,Fürstpropst,1723,1761,21892,10127
9,Koadjutor,Regensburg,1715,1716,23084,10127


Für Bischof 3807 sind Amtsart und Bistum vertauscht.

In [29]:
df_3807 = filter(:Amtsart => isequal("Regensburg"), df_prp, view = true)

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich
,String?,String,String?,String?,Int32?,Int32?,String?,String?
1,Koadjutor,Regensburg,1715,1716,23084,10127,missing,missing


In [30]:
df_3807[1, [:Amtsart, :Bistum]] = ("Koadjutor", "Regensburg")

("Koadjutor", "Regensburg")

Ämter der Ämterliste zuordnen

In [31]:
transform!(df_prp, :Amtsart => ByRow(get_role_id) => :role_id);

In [32]:
df_role_mg = filter(:role_id => ismissing, df_prp);

In [33]:
unique(df_role_mg[!, :Amtsart])[1:7]

7-element Vector{String}:
 "Fürstpropst"
 "Direktor des Konsistoriums"
 "Kurfürst-Erzbischof"
 "Apostolischer Administrator"
 "Fürstbischof"
 "Koadjutor des Bischofs"
 "Koadjutor des Kurfürst-Erzbischofs"

In diesen Fällen wird die Amtsart lediglich eingetragen in `person_role.role_name`

### Bistümer zuordnen

In [34]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese";
df_dioc = Wds.sql_df(sql);

Es sind nicht alle Ämter mit einem Bistum verknüpft, daher ist eine JOIN-Operation auf diesem Feld nicht möglich.

In [35]:
lookup_dioc = Dict(df_dioc[!,:diocese_name] .=> df_dioc[!,:diocese_id]);

In [36]:
lookup_dioc["Trier"]

30

In [37]:
get_dioc_id(name) = get(lookup_dioc, name, missing)

get_dioc_id (generic function with 1 method)

In [38]:
transform!(df_prp, :Bistum => ByRow(get_dioc_id) => :diocese_id);

In [39]:
df_dioc_mg = filter(:diocese_id => ismissing, df_prp);

In [40]:
size(df_dioc_mg)

(435, 19)

Numerische Daten eintragen

In [41]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)

num_date_begin (generic function with 1 method)

In [42]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper)

num_date_end (generic function with 1 method)

In [43]:
transform!(df_prp, :Amtsbeginn => ByRow(num_date_begin) => :num_date_begin);

┌ Warning: Could not parse 
│   s = fränkische Zeit ?
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1117


In [44]:
transform!(df_prp, :Amtsende => ByRow(num_date_end) => :num_date_end);

In [45]:
df_prp[560:567, [:ID_Bischof, :Amtsart, :Amtsbeginn, :num_date_begin, :Amtsende, :num_date_end]]

,ID_Bischof,Amtsart,Amtsbeginn,num_date_begin,Amtsende,num_date_end
,Int32?,String,String?,Int64?,String?,Int64?
1,3663,Ernannter Bischof,1249/1250,1249,missing,missing
2,3665,Elekt,1292,1292,1295,1295
3,3666,Elekt,1301,1301,missing,missing
4,3666,Elekt,1292,1292,1295,1295
5,3670,Elekt,1362,1362,1363,1363
6,3670,Bischof,1363,1363,1364,1364
7,3672,Bischof,1364,1364,1378,1378
8,3673,Bischof,1375,1375,1393,1393


Schlüssel für eine zeitliche Sortierung eintragen

In [46]:
transform!(df_prp, :Amtsbeginn => ByRow(Wds.parse_year_sort) => :date_sort_key);

┌ Warning: could not parse 
│   s = (1168) 1180
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (1014 ?) 1015
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (952)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (972)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (984)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (991)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (996)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 

In [47]:
transform!(df_prp, :Amtsende => ByRow(Wds.parse_year_sort) => :date_sort_key_replace);

┌ Warning: could not parse 
│   s = (1014 ?) 1015
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (952)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (972)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (984)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (991)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (996)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (1072)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   

In [48]:
replace_if_missing(a, b) = ismissing(a) ? b : a

replace_if_missing (generic function with 1 method)

In [49]:
transform!(df_prp, [:date_sort_key, :date_sort_key_replace] => ByRow(replace_if_missing) => :date_sort_key);

In [50]:
size(df_prp, 1), count(ismissing, df_prp.date_sort_key)

(7308, 0)

Bestehende Einträge löschen

In [51]:
table_name = "person_role";
item_type_id = 4;
sql = "DELETE FROM $(table_name) WHERE person_id IN (SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role WHERE person_id IN (SELECT id FROM item WHERE item_type_id = 4)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer, auch für die identifizierten, den Namen des Bistums in die Tabelle.  
Schreibe für alle Ämter, auch für die identifizierten, den Namen des Amts in die Tabelle.

In [52]:
columns = [
    :Bistum => :diocese_name,
    :Amtsbeginn => :date_begin,
    :Amtsende => :date_end,
    :Amt_Kommentar => :note,
    :Sort => :display_order,
    :Amtsart => :role_name,
    :person_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key,
];

In [53]:
table_dst = "person_role"
Wds.filltable!(table_dst, select(df_prp, columns))

┌ Info: Rows inserted: 7308
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


7308

Titularbistum eintragen

Identifikation über `ID_Bischof`, `Amtsart`, `Bistum`

Es gibt nur einen Eintrag, der nicht leer ist?!
``` sql
SELECT ID_Bischof, Amtsart, Bistum, COUNT(*) AS n
FROM bischof.tbl_bischoefe_gatzaemter
WHERE Titularbistum IS NOT NULL AND Titularbistum <> ''
GROUP BY ID_Bischof, Amtsart, Bistum
ORDER BY n DESC;
```

Es gibt nur einen Eintrag für `ID_Bischof` = 21596. Dieser Bischof ist aber nicht in der Bischofsliste.

In [68]:
filter(:ID_Bischof => isequal(21596), select(df_prp, :ID_Bischof, :Bistum, :Amtsart, :Titularbistum))

,ID_Bischof,Bistum,Amtsart,Titularbistum
,Int32?,String?,String?,String?


## Zeitspanne der Person

In [94]:
sql = "SELECT id, num_date_birth, num_date_death
FROM person WHERE id in (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [74]:
size(df_date)

(5171, 5)

Zeitspanne der Ämter

In [86]:
df_pr_begin_value = dropmissing(df_prp, :num_date_begin);
df_pr_begin = groupby(select(df_pr_begin_value, :person_id, :num_date_begin), :person_id);

In [87]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [61]:
size(df_pr_date_min)

(5013, 2)

In [77]:
names(df_pr_date_min)

2-element Vector{String}:
 "person_id"
 "role_date_min"

In [88]:
df_pr_end_value = dropmissing(df_prp, :num_date_end);
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id);

In [89]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [80]:
size(df_pr_date_max)

(4528, 2)

In [95]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [96]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

In [85]:
size(df_date)

(5171, 7)

In [97]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [104]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [105]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [106]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [108]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [109]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [110]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [146]:
dropmissing!(df_date, :date_min);

Trage die geänderten Daten ein 

In [120]:
sql = "CREATE TABLE date_update (id INT, date_min INT, date_max INT) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "CREATE TABLE date_update (id INT, date_min INT, date_max INT) \nENGINE=InnoDB DEFAULT CHARSET=utf8;", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

*2022-01-14* Fehler `UndefRefError: access to undefined reference` mit `Wds.filltable!`  
Füge die Zeilen einzeln ein.

In [154]:
table_name = "date_update";
# Es gibt keine fehlenden Werte
delta_msg = 400
for (i, row) in enumerate(eachrow(df_date))
    (id, date_min, date_max) = row[[:id, :date_min, :date_max]]
    sql = "INSERT INTO $(table_name) VALUES ($(id), $(date_min), $(date_max))";
    DBInterface.execute(Wds.dbwiag, sql)
    if i % delta_msg == 0
       @info "row " * i
    end
end

*2022-01-14* Schleife läuft fehlerfrei durch?!

In [159]:
sql = "UPDATE person AS p, 
(SELECT id, date_min, date_max FROM date_update) AS du
SET p.date_min = du.date_min, p.date_max = du.date_max
WHERE p.id = du.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "UPDATE person AS p, \n(SELECT id, date_min, date_max FROM date_update) AS du\nSET p.date_min = du.date_min, p.date_max = du.date_max\nWHERE p.id = du.id", 0, -1, 5037, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-14* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12